In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer, load_wine
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import ParameterGrid, GridSearchCV, RandomizedSearchCV
from IPython.display import display
pd.set_option('max_rows', 5)
pd.set_option('max_columns', 9)

## バリデーション (validation)
---
ハイパーパラメーター調整のために、学習に使用していないデータを用いてモデルを評価すること。

機械学習ではモデルのパラメーターはデータから自動的に学習するものとユーザーが設定するもの (クラス引数に与える値) がある。後者を特にハイパーパラメーターと呼ぶ。

<table class="border text-center">
    <tr class="background-dark">
        <th>種類</th>
        <th>名称</th>
        <th>具体例</th>
    </tr>
    <tr class="background-bright">
        <td>データから自動的に学習するもの</td>
        <td>パラメータ</td>
        <td>$w$ (ウェイト) や $b$ (バイアス)</td>
    </tr>
    <tr class="background-bright">
        <td>ユーザーが設定するもの</td>
        <td>ハイパーパラメーター</td>
        <td>L1 ・ L2正則化の $\alpha$</td>
    </tr>
</table>

ハイパーパラメーターの値をチューニング (調整) することで精度を向上できる。

## バリデーションの必要性
---
テストデータでの精度向上を目指してチューニングすると、ハイパーパラメーターの値にテストデータの情報が盛り込まれてしまう。  
= 手動でテストデータの情報をモデルに与え、**テストデータに対して過学習**を起こし、本当に未知のデータに対する汎化性能が測定できなくなる。

 1. あるハイパーパラメーターの値 $( \theta _{1})$ を使ってトレーニングデータ $( x_{train} ,y_{train})$ で学習
 1. テストデータ $( x_{test} ,y_{test})$ での精度 $( L_{\theta _{1}})$ を見る
 1. 別のハイパーパラメーターの値 $( \theta _{2} ,\theta _{3} ,\cdots )$ を試す
 1. テストデータで精度がよかったハイパーパラメーター $( \theta _{best})$ を採用
 1. 採用したハイパーパラメーター $( \theta _{best})$ はテストデータで精度が出るように調整されたものなので、実際に運用してみると期待した精度が出ない

そこで、ハイパーパラメーターのチューニングには、データセットをトレーニングデータ・バリデーションデータ・テストデータに分割し、

 - トレーニングデータで学習 → バリデーションデータでハイパーパラメータをチューニング → テストデータで汎化能力確認

<table class="text-center">
    <tr>
        <th colspan="5">データセット</th>
    </tr>
    <tr class="border-bottom">
        <th>トレーニングデータ</th>
        <th></th>
        <th>バリデーションデータ</th>
        <th></th>
        <th>テストデータ</th>
    </tr>
    <tr>
        <td>学習</td>
        <td>→</td>
        <td>ハイパーパラメーターのチューニング</td>
        <td>→</td>
        <td>汎化性能の確認</td>
    </tr>
</table>

という手順を踏む。

バリデーションデータを使ってハイパーパラメーターを決定した後は、そのハイパーパラメーターとデータセット全体 (トレーニングデータ+バリデーションデータ) を使ってパラメーターを学習させる。

 1. あるハイパーパラメーターの値 $( \theta _{1})$ を使ってトレーニングデータ $( x_{train} ,y_{train})$ で学習
 1. バリデーションデータ $( x_{valid} ,y_{valid})$ での精度 $( L_{\theta _{1}})$ を見る
 1. 別のハイパーパラメーターの値 $( \theta _{2} ,\theta _{3} ,\cdots )$ を試す
 1. バリデーションデータで精度がよかったハイパーパラメーター $( \theta _{best})$ を採用
 1. 採用したハイパーパラメーター $( \theta_{best})$ を使って学習用の全データ $( x_{train}+x_{valid} ,y_{train}+y_{valid})$ で学習
 1. テストデータ $( x_{test} ,y_{test})$ で汎化性能を確認

## バリデーションの種類

### ホールドアウト法
---
テストと同様にデータセットの一部からバリデーション用のデータを取り出し、トレーニングデータで学習、バリデーションデータでハイパーパラメーターの性能を評価する方法。

トレーニングデータ・バリデーションデータ・テストデータの割合は、 $6:2:2$ や $8:1:1$ などバリデーションデータとテストデータの割合を同程度にすることが多い。

#### Pythonでのホールドアウト法の実行方法
---
テストデータと同様に`sklearn.model_selection.train_test_split`を使用する。

### 交差検証法 (cross-validation)
---
訓練データを $k$ 個に分割し、1個をバリデーションデータ・残りを訓練データとして利用し性能評価、それを全ての組で行って、平均性能をそのハイパーパラメーターの性能とする。

<table class="border text-center">
    <tr class="background-dark">
        <th colspan="6">パラメータ $\theta _{1}$ の学習</th>
    </tr>
    <tr class="background-dark">
        <td></td>
        <td>データセット1</td>
        <td>データセット2</td>
        <td>…</td>
        <td>データセットk</td>
        <td>平均</td>
    </tr>
    <tr class="background-bright">
        <td>1回目</td>
        <td>バリデーションデータ</td>
        <td>トレーニングデータ</td>
        <td>…</td>
        <td>トレーニングデータ</td>
        <td rowspan="4">$\theta _{1}$ の性能</td>
    </tr>
    <tr class="background-bright">
        <td>2回目</td>
        <td>トレーニングデータ</td>
        <td>バリデーションデータ</td>
        <td>…</td>
        <td>トレーニングデータ</td>
    </tr>
    <tr class="background-bright">
        <td colspan="5">$\vdots$</td>
    </tr>
    <tr class="background-bright">
        <td>k回目</td>
        <td>トレーニングデータ</td>
        <td>トレーニングデータ</td>
        <td>…</td>
        <td>バリデーションデータ</td>
    </tr>
    <tr class="border-none background-default">
        <td colspan="6"></td>
    </tr>
    <tr class="background-dark">
        <th colspan="6">パラメータ $\theta _{2}$ の学習</th>
    </tr>
    <tr class="background-dark">
        <td></td>
        <td>データセット1</td>
        <td>データセット2</td>
        <td>…</td>
        <td>データセットk</td>
        <td>平均</td>
    </tr>
    <tr class="background-bright">
        <td>1回目</td>
        <td>バリデーションデータ</td>
        <td>トレーニングデータ</td>
        <td>…</td>
        <td>トレーニングデータ</td>
        <td rowspan="4">$\theta _{2}$ の性能</td>
    </tr>
    <tr class="background-bright">
        <td>2回目</td>
        <td>トレーニングデータ</td>
        <td>バリデーションデータ</td>
        <td>…</td>
        <td>トレーニングデータ</td>
    </tr>
    <tr class="background-bright">
        <td colspan="5">$\vdots$</td>
    </tr>
    <tr class="background-bright">
        <td>k回目</td>
        <td>トレーニングデータ</td>
        <td>トレーニングデータ</td>
        <td>…</td>
        <td>バリデーションデータ</td>
    </tr>
    <tr class="border-none background-default">
        <td colspan="6">$\vdots$</td>
    </tr>
</table>

 - トレーニングデータとバリデーションデータの分け方による誤差に強い
 - 各ハイパーパラメーターごとに $k$ 回学習を行うので、実行にかかる時間が長くなる

特に分類問題で完全にランダムにデータを分割すると、各グループでクラスの偏りが発生する可能性があるので、元データのクラス比率を維持しながら分割する層化 k 分割交差検証 (stratified k-fold cross-validation) を用いることも多い。

極端にデータが少ない場合にはバリデーションデータとして 1 サンプルのみを使用する LOO (Leave One Out) も使われることがある。

#### クロスバリデーションとテスト
---
クロスバリデーションを実施する際もテストを行うのが望ましいが、計算コストの高いクロスバリデーションは**データが少ない場合に使われることが多く**、テストデータを十分に確保できない。また、 1 つのハイパーパラメーターに対して複数のデータ分割で評価しているので、ホールドアウト法より過学習を起こしにくい。  
そのため、クロスバリデーションを実施する場合にはテストは行わないことも多い。

#### Pythonでのクロスバリデーションの実行方法
---
`sklearn.model_selection.cross_validate`や各モデル名の後に CV のついたもの (`RidgeCV`など) を使用する。

In [2]:
loader = load_wine()
wine = pd.DataFrame(np.column_stack([loader.data, loader.target]), columns=list(loader.feature_names)+['target'])
print('wine')
display(wine)

wine


,alcohol,malic_acid,ash,alcalinity_of_ash,...,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,...,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,...,1.05,3.40,1050.0,0.0
...,...,...,...,...,...,...,...,...,...
176,13.17,2.59,2.37,20.0,...,0.60,1.62,840.0,2.0
177,14.13,4.10,2.74,24.5,...,0.61,1.60,560.0,2.0


In [3]:
cross_validate??

In [4]:
x = wine.iloc[:, :-1]
y = wine.iloc[:, -1]
model = DecisionTreeClassifier()
cross_validate(model, x, y, cv=5, n_jobs=-1)

{'fit_time': array([0.00326562, 0.00321031, 0.03324509, 0.00886345, 0.00340986]),
 'score_time': array([0.00169373, 0.00252652, 0.00148702, 0.00944495, 0.00138354]),
 'test_score': array([0.86486486, 0.86111111, 0.94444444, 0.91428571, 0.82352941])}

## ハイパーパラメーターサーチ
---
様々なハイパーパラメーターを試して、精度のいいものを探すこと。

### グリッドサーチ
---
与えられたハイパーパラメーターの値候補の全組み合わせを試して、最も精度のよい組み合わせを探す方法。

#### Pythonでのグリッドサーチの実行方法
---
`sklearn.model_selection.GridSearchCV`を使用する。`sklearn.model_selection.ParameterGrid`で探索するハイパーパラメーターの全組み合わせを作成しておく必要がある。

In [5]:
ParameterGrid??

In [6]:
params = dict(criterion=['gini', 'entropy'], max_depth=[3, 4, 5])
grid = ParameterGrid(params)
list(grid)

[{'criterion': 'gini', 'max_depth': 3},
 {'criterion': 'gini', 'max_depth': 4},
 {'criterion': 'gini', 'max_depth': 5},
 {'criterion': 'entropy', 'max_depth': 3},
 {'criterion': 'entropy', 'max_depth': 4},
 {'criterion': 'entropy', 'max_depth': 5}]

In [7]:
GridSearchCV??

In [8]:
gs = GridSearchCV(model, params, n_jobs=-1, cv=5)
gs.fit(x, y)
gs.cv_results_

/root/local/python-3.7.4/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'mean_fit_time': array([0.00338502, 0.00492487, 0.00315108, 0.00726948, 0.00485044,
        0.00324054]),
 'std_fit_time': array([0.0003883 , 0.00181055, 0.00035019, 0.00870925, 0.00194904,
        0.00027113]),
 'mean_score_time': array([0.00163679, 0.00237145, 0.00164824, 0.00259037, 0.00174861,
        0.00170245]),
 'std_score_time': array([0.00015081, 0.00157183, 0.00024042, 0.00219463, 0.00066622,
        0.00041292]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'entropy', 'entropy',
                    'entropy'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[3, 4, 5, 3, 4, 5],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'criterion': 'gini', 'max_depth': 3},
  {'criterion': 'gini', 'max_depth': 4},
  {'criterion': 'gini', 'max_depth': 5},
  {'criterion': 'entropy', 'max_

In [9]:
gs.best_params_

{'criterion': 'entropy', 'max_depth': 5}

### ランダムサーチ
---
与えられたハイパーパラメーターの値候補からランダムに組み合わせて、精度のよい組み合わせを探す方法。  
複数のハイパーパラメーターのうち、精度に大きく影響するものは少数であったり、どのハイパーパラメーターをどのくらいの範囲で探索するのが効率的かわからなかったりするので、グリッドサーチを実施する前に絞り込みに利用したりする。

#### Pythonでのランダムサーチの実行方法
---
`sklearn.model_selection.RandomizedSearchCV`を使用する。

In [10]:
RandomizedSearchCV??

In [11]:
rs = RandomizedSearchCV(model, params, n_iter=3, n_jobs=-1, cv=5, random_state=1234)
rs.fit(x, y)
rs.cv_results_

/root/local/python-3.7.4/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


{'mean_fit_time': array([0.00629029, 0.0054584 , 0.00372019]),
 'std_fit_time': array([0.00213839, 0.0007098 , 0.00036564]),
 'mean_score_time': array([0.00295606, 0.00226474, 0.00140395]),
 'std_score_time': array([0.00170592, 0.00069231, 0.0001398 ]),
 'param_max_depth': masked_array(data=[5, 4, 5],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'param_criterion': masked_array(data=['gini', 'gini', 'entropy'],
              mask=[False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 5, 'criterion': 'gini'},
  {'max_depth': 4, 'criterion': 'gini'},
  {'max_depth': 5, 'criterion': 'entropy'}],
 'split0_test_score': array([0.78378378, 0.81081081, 0.81081081]),
 'split1_test_score': array([0.80555556, 0.83333333, 0.91666667]),
 'split2_test_score': array([0.88888889, 0.94444444, 0.88888889]),
 'split3_test_score': array([0.91428571, 0.91428571, 0.97142857]),
 'split4_test_score': array([0.823529

In [12]:
rs.best_params_

{'max_depth': 4, 'criterion': 'gini'}